# Predicting Freight Costs for Antiretroviral (AVR) and HIV Lab Shipments
- Abby Lloyd
- Capstone: MS in Data Analytics
- Northwest Missouri State University
- abbylloyd03@gmail.com

## Part One: Import & Inspect Data

### 1. Import Data

In [ ]:
# Import necessary packages
import pandas as pd
from sodapy import Socrata

# Access and load data from API
# API documentation: https://dev.socrata.com/foundry/data.usaid.gov/a3rc-nmf6
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.usaid.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.usaid.gov,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("a3rc-nmf6", limit=20000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

### 2. Inspect Data

In [9]:
results_df.describe()

id project_code              pq        po_so     asn_dn  \
count   10324        10324           10324        10324      10324   
unique  10324          142            1237         6233       7030   
top         1   116-ZA-T30  Pre-PQ Process  SCMS-199289  ASN-19166   
freq        1          768            2681           67         54   

             country managed_by fulfill_via vendor_inco_term shipment_mode  \
count          10324      10324       10324            10324          9964   
unique            43          4           2                8             4   
top     South Africa   PMO - US    From RDC   N/A - From RDC           Air   
freq            1406      10265        5404             5404          6113   

        ... line_item_quantity line_item_value pack_price unit_price  \
count   ...              10324           10324      10324      10324   
unique  ...               5065            8741       1175        183   
top     ...              10000        200000.0       32.0       0.04   
freq    ...                 93              29        368        713   

               manufacturing_site first_line_designation  \
count                       10324                  10324   
unique                         88                      2   
top     Aurobindo Unit III, India                   True   
freq                         3172                   7030   

                  weight_kilograms                    freight_cost_usd dosage  \
count                        10324                               10324   8588   
unique                        4688                                6733     54   
top     Weight Captured Separately  Freight Included in Commodity Cost  300mg   
freq                          1507                                1442    990   

       line_item_insurance_usd  
count                    10037  
unique                    6722  
top                        0.0  
freq                        54  

[4 rows x 33 columns]

In [10]:
results_df.head()

id project_code              pq    po_so  asn_dn        country managed_by  \
0   1   100-CI-T01  Pre-PQ Process   SCMS-4   ASN-8  Côte d'Ivoire   PMO - US   
1   3   108-VN-T01  Pre-PQ Process  SCMS-13  ASN-85        Vietnam   PMO - US   
2   4   100-CI-T01  Pre-PQ Process  SCMS-20  ASN-14  Côte d'Ivoire   PMO - US   
3  15   108-VN-T01  Pre-PQ Process  SCMS-78  ASN-50        Vietnam   PMO - US   
4  16   108-VN-T01  Pre-PQ Process  SCMS-81  ASN-55        Vietnam   PMO - US   

   fulfill_via vendor_inco_term shipment_mode  ... line_item_quantity  \
0  Direct Drop              EXW           Air  ...                 19   
1  Direct Drop              EXW           Air  ...               1000   
2  Direct Drop              FCA           Air  ...                500   
3  Direct Drop              EXW           Air  ...              31920   
4  Direct Drop              EXW           Air  ...              38000   

  line_item_value pack_price unit_price             manufacturing_site  \
0           551.0       29.0       0.97     Ranbaxy Fine Chemicals LTD   
1          6200.0        6.2       0.03      Aurobindo Unit III, India   
2         40000.0       80.0        0.8  ABBVIE GmbH & Co.KG Wiesbaden   
3        127360.8       3.99       0.07  Ranbaxy, Paonta Shahib, India   
4        121600.0        3.2       0.05      Aurobindo Unit III, India   

  first_line_designation weight_kilograms freight_cost_usd   dosage  \
0                   True               13           780.34      NaN   
1                   True              358           4521.5  10mg/ml   
2                   True              171          1653.78      NaN   
3                   True             1855         16007.06    150mg   
4                   True             7590         45450.08     30mg   

  line_item_insurance_usd  
0                     NaN  
1                     NaN  
2                     NaN  
3                     NaN  
4                     NaN  

[5 rows x 33 columns]

## Part Two: Clean the Data

### 1. Choose Countries and Manufacturing Sites to Focus On
- The original data set contained shipments to 43 unique countries. This research will focus on the the countries that recieved the most shipments. 
- After the top countries have been determined, shipments to all other countries will be removed from the dataset.
- Similarly, the top manufacturing sites that sent shipments to these countries will be determined, and shipments from all other sites will be removed.

In [69]:
# Strip any extra whitespace from the values in country column
results_df['country'].str.strip()

# Return value counts (%) for country column
results_df['country'].value_counts(normalize=True).nlargest(20)

South Africa     0.136188
Nigeria          0.115653
Côte d'Ivoire    0.104901
Uganda           0.075455
Vietnam          0.066641
Zambia           0.066157
Haiti            0.063444
Mozambique       0.061120
Zimbabwe         0.052112
Tanzania         0.050271
Rwanda           0.041651
Congo, DRC       0.032255
Guyana           0.022956
Ethiopia         0.020922
South Sudan      0.015885
Kenya            0.010752
Burundi          0.009492
Namibia          0.009202
Cameroon         0.007265
Botswana         0.006780
Name: country, dtype: float64

In [64]:
# Remove rows not included in the top countries and inspect new data set
countries_to_include = ['South Africa', 'Nigeria', 'Côte d\'Ivoire', 'Uganda', 'Vietnam', 
                     'Zambia', 'Haiti', 'Mozambique', 'Zimbabwe', 'Tanzania']

df_top_countries = results_df[results_df['country'].isin(countries_to_include)]
df_top_countries.describe()

id project_code              pq        po_so     asn_dn  \
count   8176         8176            8176         8176       8176   
unique  8176           53             800         4847       5494   
top        1   116-ZA-T30  Pre-PQ Process  SCMS-199289  ASN-19166   
freq       1          768            2238           67         54   

             country managed_by fulfill_via vendor_inco_term shipment_mode  \
count           8176       8176        8176             8176          7844   
unique            10          3           2                8             4   
top     South Africa   PMO - US    From RDC   N/A - From RDC           Air   
freq            1406       8118        4415             4415          4176   

        ... line_item_quantity line_item_value pack_price unit_price  \
count   ...               8176            8176       8176       8176   
unique  ...               4379            7166       1091        159   
top     ...              10000        200000.0       80.0       0.04   
freq    ...                 84              29        201        563   

               manufacturing_site first_line_designation  \
count                        8176                   8176   
unique                         77                      2   
top     Aurobindo Unit III, India                   True   
freq                         2694                   5494   

                  weight_kilograms                    freight_cost_usd dosage  \
count                         8176                                8176   7298   
unique                        4071                                5140     53   
top     Weight Captured Separately  Freight Included in Commodity Cost  300mg   
freq                          1366                                1394    830   

       line_item_insurance_usd  
count                     7925  
unique                    5780  
top                       0.07  
freq                        27  

[4 rows x 33 columns]

In [63]:
# Strip any extra whitespace from the values in manufacturing_site column
df_top_countries['manufacturing_site'].str.strip()

# Return value counts (%) for manufacturing_site column
df_top_countries['manufacturing_site'].value_counts(normalize=True).nlargest(20)

Aurobindo Unit III, India                                                   0.329501
Mylan (formerly Matrix) Nashik                                              0.143713
Hetero Unit III Hyderabad IN                                                0.086473
Cipla, Goa, India                                                           0.073141
Strides, Bangalore, India.                                                  0.053327
ABBVIE Ludwigshafen Germany                                                 0.040484
Alere Medical Co., Ltd.                                                     0.034124
Inverness Japan                                                             0.022994
ABBVIE (Abbott) Logis. UK                                                   0.022750
BMS Meymac, France                                                          0.017613
Aspen-OSD, Port Elizabeth, SA                                               0.016512
Trinity Biotech, Plc                                             

In [68]:
# Remove rows not included in the top manufacturing sites and inspect new data set
man_site_to_include = ['Aurobindo Unit III, India', 'Mylan (formerly Matrix) Nashik', 'Hetero Unit III Hyderabad IN',
                       'Cipla, Goa, India', 'Strides, Bangalore, India.', 'ABBVIE Ludwigshafen Germany']

df_top_countries_top_man = df_top_countries[df_top_countries['manufacturing_site'].isin(man_site_to_include)]
df_top_countries_top_man.describe()

id project_code              pq        po_so     asn_dn  \
count   5941         5941            5941         5941       5941   
unique  5941           43             513         3544       3892   
top        3   151-NG-T30  Pre-PQ Process  SCMS-199289  ASN-19166   
freq       1          589            1369           67         33   

             country managed_by fulfill_via vendor_inco_term shipment_mode  \
count           5941       5941        5941             5941          5755   
unique            10          2           2                5             4   
top     South Africa   PMO - US    From RDC   N/A - From RDC           Air   
freq             950       5907        3889             3889          2662   

        ... line_item_quantity line_item_value pack_price unit_price  \
count   ...               5941            5941       5941       5941   
unique  ...               3679            5419        692         93   
top     ...              50000         16000.0      11.22       0.04   
freq    ...                 57              11        127        519   

               manufacturing_site first_line_designation  \
count                        5941                   5941   
unique                          6                      2   
top     Aurobindo Unit III, India                   True   
freq                         2694                   3784   

                  weight_kilograms                    freight_cost_usd dosage  \
count                         5941                                5941   5941   
unique                        3469                                3749     39   
top     Weight Captured Separately  Freight Included in Commodity Cost  300mg   
freq                           782                                 847    711   

       line_item_insurance_usd  
count                     5818  
unique                    4562  
top                       0.07  
freq                        16  

[4 rows x 33 columns]

### 2. Remove rows that do not include freight cost

In [67]:
# Strip any extra whitespace from the values in freight_cost_usd column
df_top_countries_top_man['freight_cost_usd'].str.strip()

# Remove rows that do not include freight costs and inspect new data set
freight_price_not_listed = ['Freight Included in Commodity Cost', 'Invoiced Separately']
final_df = df_top_countries_top_man[~df_top_countries_top_man['freight_cost_usd'].isin(freight_price_not_listed)]
final_df.describe()

id project_code              pq     po_so   asn_dn        country  \
count   5014         5014            5014      5014     5014           5014   
unique  5014           42             485      3244     3515             10   
top        3   104-CI-T30  Pre-PQ Process  SO-44400  DN-3015  Côte d'Ivoire   
freq       1          566            1260        13       13            763   

       managed_by fulfill_via vendor_inco_term shipment_mode  ...  \
count        5014        5014             5014          4830  ...   
unique          1           2                5             4  ...   
top      PMO - US    From RDC   N/A - From RDC           Air  ...   
freq         5014        3688             3688          2479  ...   

       line_item_quantity line_item_value pack_price unit_price  \
count                5014            5014       5014       5014   
unique               3390            4590        482         70   
top                 50000         16000.0      11.22       0.04   
freq                   53              11        111        492   

               manufacturing_site first_line_designation  \
count                        5014                   5014   
unique                          6                      2   
top     Aurobindo Unit III, India                   True   
freq                         2092                   3412   

               weight_kilograms freight_cost_usd dosage  \
count                      5014             5014   5014   
unique                     3408             3747     39   
top     See DN-3015 (ID#:82554)           9736.1  300mg   
freq                         12               36    567   

       line_item_insurance_usd  
count                     4896  
unique                    4174  
top                       5.12  
freq                        10  

[4 rows x 33 columns]

### 3. Remove columns that are not neccesary